In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!ls "/content/gdrive/My Drive/Code for San Jose"

 AlisaViejo_Granicus_01-20-2021.pdf	 Example2.gdoc
 AlisaViejo_Granicus.pdf		 Example2.txt
'Copy of pdf_keyword_extraction.ipynb'	 pdf_keyword_extraction.ipynb
'Example2 (1).gdoc'			 topic_modeling.ipynb


In [3]:
!pip install PyPDF2

In [4]:
!pip install textract

In [5]:
!pip install regex

Part 1: Extract Text from PDF for Processing.

In [6]:
import pandas as pd
import numpy as np
import PyPDF2
import textract
import re

In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
filename="/content/gdrive/My Drive/Code for San Jose/AlisaViejo_Granicus.pdf"

In [9]:
pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from our PDF file.

In [10]:
text = text.encode('ascii','ignore').lower() #Lowercasing each word

In [11]:
text = text.decode('ISO-8859-1')

In [12]:
text

'city of aliso viejoagenda itemdate:october 21, 2020to:mayor and city council  from:omar dadabhoy, community development director  subject:ordinance amending title 15 of the city of aliso viejo \nmunicipal code relating to medical uses inthe town \ncenter commercial (ct) zonerecommended actions:\n1.find that the proposed amendments are exempt from the california environmental \nquality act (ceqa) pursuant to section 15061(b)(3) of the state ceqa guidelines, \nbecause it will not result in a direct or reasonable foreseeable indirect physical change \nin theenvironment as there is no possibility that it may have a significant effect on the \nenvironment and because it is not a project as defined in section 15378 of the ceqa \nguidelines. 2.introduce an ordinance (attachment 1) entitled: \nan ordinance of the city council of the city of aliso viejo, \ncalifornia, amending table 15.18.020 of section 15.18.020 of chapter \n15.18 of the aliso viejo municipal code, permitted uses in \nnonresi

In [16]:
def lemmatize_stemming(text):
  stemmer = SnowballStemmer('english')
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [17]:
print('original document: ')
words = []
for word in text.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(text))

original document: 
['city', 'of', 'aliso', 'viejoagenda', 'itemdate:october', '21,', '2020to:mayor', 'and', 'city', 'council', '', 'from:omar', 'dadabhoy,', 'community', 'development', 'director', '', 'subject:ordinance', 'amending', 'title', '15', 'of', 'the', 'city', 'of', 'aliso', 'viejo', '\nmunicipal', 'code', 'relating', 'to', 'medical', 'uses', 'inthe', 'town', '\ncenter', 'commercial', '(ct)', 'zonerecommended', 'actions:\n1.find', 'that', 'the', 'proposed', 'amendments', 'are', 'exempt', 'from', 'the', 'california', 'environmental', '\nquality', 'act', '(ceqa)', 'pursuant', 'to', 'section', '15061(b)(3)', 'of', 'the', 'state', 'ceqa', 'guidelines,', '\nbecause', 'it', 'will', 'not', 'result', 'in', 'a', 'direct', 'or', 'reasonable', 'foreseeable', 'indirect', 'physical', 'change', '\nin', 'theenvironment', 'as', 'there', 'is', 'no', 'possibility', 'that', 'it', 'may', 'have', 'a', 'significant', 'effect', 'on', 'the', '\nenvironment', 'and', 'because', 'it', 'is', 'not', 'a',

In [18]:
lem_stem_words = list()
lem_stem_words.append(words)

In [21]:
df = pd.DataFrame(list(set(words)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows

In [22]:
df.head(10)

,keywords
0,
1,only
2,exempt
3,\nhearing
4,actions:\n1.find
5,(attachment
6,which
7,building
8,"17,171"
9,square


In [23]:
df.to_csv('out.csv', index=False)